1. train.csv : 2019년 01월 01일부터 2023년 03월 03일까지의 유통된 품목의 가격 데이터
- item: 품목 코드 - TG : 감귤, BC : 브로콜리, RD : 무, CR : 당근, CB : 양배추
- corporation : 유통 법인 코드 - 법인 A~F
- location : 지역 코드 - J : 제주도 제주시, S : 제주도 서귀포시
- supply(kg) : 유통된 물량, kg 단위
- price(원/kg) : 유통된 품목들의 kg 마다의 가격, 원 단위

2. international_trade.csv : 관련 품목 수출입 정보
- 중량 단위 kg
- 금액 단위 천 달러

3. test.csv : 2023년 03월 04일부터 2023년 03월 31일까지의 데이터

4. sample_submission.csv : 제출을 위한 양식
- 2023년 03월 04일부터 2023년 03월 31일까지의 price(원/kg)을 예측
- ID는 품목, 유통 법인, 지역 코드로 구성된 식별자
- 해당 ID에 맞춰 price(원/kg) 예측값을 answer 컬럼에 기입해야 함

In [1]:
#import
import random
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings(action='ignore') 

C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
#Fixed Random-Seed
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYHONAHSEED"]=str(seed)
    np.random.seed(seed)
seed_everything(42) #시드고정

In [3]:
#Load Data
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [4]:
#Data preProcessing
#시계열의 특성을 반영하기 위해 timestamp를 월, 일, 시간으로 나눔
train_df['year'] = train_df['timestamp'].apply(lambda x : int(x[0:4]))
train_df['month'] = train_df['timestamp'].apply(lambda x: int(x[5:7]))
train_df['day'] = train_df['timestamp'].apply(lambda x: int(x[8:10]))

test_df['year']=test_df['timestamp'].apply(lambda x:int(x[0:4]))
test_df['month'] = test_df['timestamp'].apply(lambda x: int(x[5:7]))
test_df['day'] = test_df['timestamp'].apply(lambda x: int(x[8:10]))

In [5]:
#학습에 사용하지 않을 변수들을 제거
train_x = train_df.drop(columns = ['ID','timestamp','supply(kg)','price(원/kg)'])
train_y = train_df['price(원/kg)']

test_x = test_df.drop(columns=["ID","timestamp"])

In [6]:
#질적 변수들 수치화
qual_col = ['item','corporation','location']
for i in qual_col:
    le = LabelEncoder()
    train_x[i]=le.fit_transform(train_x[i])
    test_x[i]=le.transform(test_x[i]) #test데이터에 대해서 fit하는 것을 data leakage에 해당
    
print("Done")

Done


In [7]:
#Regression Model Fit
model = RandomForestRegressor()
model.fit(train_x, train_y)

RandomForestRegressor()

In [9]:
#Inference
preds = model.predict(test_x)

In [11]:
#submission
submission = pd.read_csv("data/sample_submission.csv")
submission

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


In [12]:
submission['answer']=preds
submission

,ID,answer
0,TG_A_J_20230304,3636.64
1,TG_A_J_20230305,4109.28
2,TG_A_J_20230306,401.82
3,TG_A_J_20230307,3277.93
4,TG_A_J_20230308,3067.03
...,...,...
1087,RD_F_J_20230327,271.27
1088,RD_F_J_20230328,441.28
1089,RD_F_J_20230329,439.31
1090,RD_F_J_20230330,436.00


In [13]:
submission.to_csv('./baseline_submission.csv', index=False)